In [56]:
# Import the necessary dependencies

# Operating System
import os

# Numpy, Pandas and Scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix, save_npz, load_npz

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Model Evaluation
from evaluation import evaluate

# BLU12 - Learning Notebook - Workflow
## Introduction
This week we are going to simulate the real-life environment of the Hackathon! We will provide you with a dataset, some tips and you are going to train, validate and predict with a Recommender System using all the knowledge acquired throughout this specialization. This exercise is very similar to the Hackathon of the previous year.

## Context
You have finished the Academy and you are hired as a Data Scientist in Recommentunify *(ignore the horrible company name, it works and we have more important stuff to discuss)*.

First week and the job and the CEO of the next-unicorn-startup that hired you pitches your first task for the company. NASA has discovered that "Despacito 2" is on the making and you need to **urgently avoid this scenario.**

<img src="./media/despacito_nasa.jpg" alt="Miss me yet?" width="500"/>

He doesn't care how you do it or how it works, just that you use fancy trendy eye-catching mind-blowing AI *stuff* to promote Recommentunify's name in the Data Science industry.

## Technical Task
After the pitch from the CEO, you talk with your Lead Data Scientist and you get the tasks needed to fulfil the job. The company collected some data on the users, you will need to create the Recommender System that is effective enough so that your users will not need a 2nd version of a Despacito.

Your Lead DS keeps some of the data as a test set (in which you will estimate production performance) and gives you the remaining data for yourself.

### Technical Details - Type Hints
Since this specialization focuses on the workflow, we will add a cool new feature of more recent versions of Python called "Type Hints". Hints allow you to document the code "within itself" and allow for new users and reviewers to better understand what you are doing (consider your future self as a new user when you have to pick up code that you've created a long time ago). This is basically adding the type of the argument after the argument name in the function. You can also hint what should be the return type. If you do not use anything else, Python will not enforce the argument type (will acept as it is), but this is still useful for documentation and debugging.

```
def add_two(x: int) -> int: return x+2

print(add_two(2))     # receives int and returns 4, which is an int
print(add_two(2.2))   # receives float and returns 4.2 , which is also float (still works)
```

### Technical Details - Docstrings
You can make use of the [Docstrings](https://en.wikipedia.org/wiki/Docstring) to better document your code. There are several styles available. We will use the "Google Style" for this notebook.

As an example, you can document in a function its overall behaviour (what is used for), what are the expected parameters and returns (along with their types), as well as defining the TODOs for the difficult tasks you are too lazy to code yourself *(future self will be wiser to pick-up on this).*

## Step -1: Understanding the data
You have available under the `./data/` folder some files that you can use for building the Recommender System. As a good scientist, the first step is to validate that the data you have will be good enough to proceed.

The files we have available for the **training** stage are:
* `train_play_counts.txt` has the listening history for each user
* `song_tag.csv`: has the relationship between tags and songs
* `songs.txt`: has the correspondence between the song_id (a unique id, eg: SOAAADD12AB018A9DD) and the song_index (an integer index, eg: 1)
* `tags.csv`: has the correspondence between the tag_id (a tag name, eg: classic rock) and the tag_index (an integer index, eg: 1)



For the **test** stage, we have:
* `test_users.csv`: has the user_ids for which a recommendation should be produced
* `example_output.csv`: has an example of an output (just for a couple of users)

The best approach is to look at the raw files and print out the first rows of each file just to get a overall sense of what we have.

*Note for Windows users: remember that you need to convert the head command into the Windows equivalent. Tip: you can use what you learned in Data Wrangling Learning Unit!*

In [ ]:
print("train_play_counts.txt \n")
!head -3 data/train_play_counts.txt

train_play_counts.txt 



In [3]:
print("song_tag.csv\n")
!head -4 data/song_tag.csv

song_tag.csv

song_index,tag_index,val
254229,206,100.0
254229,1125,66.0
254229,582,66.0


In [4]:
print("songs.txt \n")
!head -3 data/songs.txt

songs.txt 

SOAAADD12AB018A9DD 1
SOAAADE12A6D4F80CC 2
SOAAADF12A8C13DF62 3


In [5]:
print("tags.csv \n")
!head -4 data/tags.csv

tags.csv 

tag_id,tag_index
classic rock,1
Progressive rock,2
blues,3


In [6]:
print("test_users.csv \n")
!head -3 data/test_users.csv

test_users.csv 

56d985c92960b98ad76a48b10a062b0cd86795bf
0102d1549242d159df98f333aee4041f96f37e98
991411f0dca94f348c7bd3eae93b6e6c061605f1


In [7]:
print("example_output.csv \n")
!head -1 data/example_output.csv

example_output.csv 

ba986b9277b96cd6de07dd07be8362b67b764dd4,12985,304842,293304,414,25150,314086,68212,217471,14397,177172,54386,315812,319911,243769,302247,86545,2078,16220,310418,302369,289658,221730,333259,311604,52478,307202,329834,25323,337625,277126,87433,170541,212702,24825,212017,348629,127365,241705,183796,341456,170222,207916,384072,132813,107193,188549,362714,138672,21771,339689,38941,177486,244143,285387,157386,307140,126831,302031,181854,192716,334187,57500,165401,305991,207935,351764,105434,45592,187097,311262,266750,109807,90822,311702,208383,52176,243307,97972,19682,353035,30292,72314,105694,108137,335634,55364,84395,147076,261596,49469,39842,245936,69165,202995,49781,142602,223714,122591,333064,149655,11147,69227,252967,8402,180413,291341,321987,79814,167248,168430,30643,36589,362643,165753,247135,378274,172483,353700,93787,177574,150116,250930,245012,129495,307414,315425,340072,334240,114624,39401,183790,348308,47391,242558,186377,100897,248603,142745,261901,162388,

## Step 0: Load the Data
After validating that the data we have is good enough, we start with building the ratings matrix. Our strategy for the first model is to get a non-personalized system as a baseline. This allows us to both predict for existing users (no cold-start problem) and for new users.

Note: since the `song_tag.csv` is heavy, it was stored in a zip file which you will need to unzip.

## Step 0.1: Load the Train and Test files

In [57]:
def read_users_history() -> pd.DataFrame:
    """Imports the listening history for each user.
    
    Returns:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
    """
    path = os.path.join('data', 'train_play_counts.txt')
    data = pd.read_csv(path, names=['user', 'song_id', 'rating'], sep='\t')
    return data

data = read_users_history()
data.head()

,user,song_id,rating
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [58]:
# How many ratings do we have in total?
# Tip: The ":," at the end of the f-string adds the thousand separator.
print(f"We have {len(data):,} ratings in total.")

We have 100,000 ratings in total.


In [59]:
def read_test_users() -> pd.DataFrame:
    """Imports the list of users for which we need to predict.
    
    Returns:
        users_to_pred (pd.DataFrame): DataFrame with the users for which we will recommend songs. 
    """

    path = os.path.join('data', 'test_users.csv')
    users_to_pred_ = pd.read_csv(path, names=['users to recommend songs'])
    
    return users_to_pred_


users_to_pred = read_test_users()
users_to_pred.head()

,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,0102d1549242d159df98f333aee4041f96f37e98
2,991411f0dca94f348c7bd3eae93b6e6c061605f1
3,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
4,323fbb28144eefa3eabfa22bd310dfb0713de80d


In [60]:
# For how many users are we recommending stuff?
print(f"We have {len(users_to_pred)} users in need for better songs!")

We have 1000 users in need for better songs!


## Step 0.2: Compare the train and test files

In [61]:
# And for how many users we already know something?
# For the 1000 users in our test set, we have data for 700 in the original listening history.
# For the remaining 300 we will have to use non-personalized strategies.
users_to_pred.isin(data.user.values).sum()

users to recommend songs    700
dtype: int64

In [62]:
def get_indices_from_users_to_pred(users_to_pred: pd.DataFrame, data: pd.DataFrame):
    """Get the indices of users_to_pred for which we have data and for which we don't.
    
    Args:
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        data (pd.DataFrame): Original of listening history for the users.
        
    Returns:
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
        index_users_not_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's without training data.
        
    """
    index_users_in_data = users_to_pred[users_to_pred.isin(data.user.values).values].index
    index_users_not_in_data = users_to_pred[~users_to_pred.isin(data.user.values).values].index
    
    return index_users_in_data, index_users_not_in_data


index_users_in_data, index_users_not_in_data = get_indices_from_users_to_pred(users_to_pred, data)

In [63]:
# For further inspection, we advise you to look at the objects themselves.
print(f"The index for users which we have training data has length of {len(index_users_in_data)}.")
print(f"The index for users which we don't have training data has length of {len(index_users_not_in_data)}.")

The index for users which we have training data has length of 700.
The index for users which we don't have training data has length of 300.


In [64]:
def get_users_to_pred_by_index(users_to_pred, index_users_in_data):
    """DataFrame with user_id's in test set for for which we have training data.

    Args: 
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
    Returns:
        users_in_data (pd.DataFrame): Dataframe containing the list of user_id's with training data.
    
    """
    return users_to_pred.iloc[index_users_in_data].reset_index(drop=True)

# Get the test users with training data
test_users_in_data = get_users_to_pred_by_index(users_to_pred, index_users_in_data)
print(test_users_in_data.shape)
test_users_in_data.head()

(700, 1)


,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,991411f0dca94f348c7bd3eae93b6e6c061605f1
2,323fbb28144eefa3eabfa22bd310dfb0713de80d
3,55c750f0951ca1021b26c0e758660bb8a2c49d3a
4,b458e3d697276a93aa6926caf1ff08e875933940


In [65]:
# Get the test users without training data
test_users_not_in_data = get_users_to_pred_by_index(users_to_pred, index_users_not_in_data)
print(test_users_not_in_data.shape)
test_users_not_in_data.head()

(300, 1)


,users to recommend songs
0,0102d1549242d159df98f333aee4041f96f37e98
1,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
2,02e4174f754ee4546f4c4438a4bea6ed877c6c4f
3,03070a3284db565b2e67f8ad01ce96c31c9986dd
4,01dd0b2bb39643b8925a672105819d599f827c87


## Step 0.3: Create the Ratings matrix

In [66]:
def make_ratings(data: pd.DataFrame) -> csr_matrix:
    """Creates the ratings matrix of listening history.
    
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        ratings (csr_matrix): ratings matrix with shape (n_users, n_items)
        
    TODO:
        * Add the shape as an optional parameter.
        
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    
    return R_


R = make_ratings(data)
R

<7526x41194 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [67]:
# Just for mental (in)sanity, let's match the info of the matrix to what is printed in the previous cell.
print(f"The shape is {R.shape}")
print(f"The dtypes of the elements are {R.dtype}")
print(f"The number of stored elements is {R.nnz}")
print(f"The type of the matrix is {type(R)}")

The shape is (7526, 41194)
The dtypes of the elements are int64
The number of stored elements is 100000
The type of the matrix is <class 'scipy.sparse.csr.csr_matrix'>


In [68]:
# Let's store a Series with the unique user id's that we have in the original data.
def get_unique_users(data):
    """Get unique users in training data.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        unique_users (pd.DataFrame): DataFrame of one column with unique users in training data.
    
    """
    return pd.DataFrame(np.unique(data.iloc[:, 0].values), columns=["users to recommend songs"])


unique_users_training_data = get_unique_users(data)
unique_users_training_data.head()

,users to recommend songs
0,0011d5f4fb02ff276763d385c3f2ded2b00ad94a
1,002511b392561fc1d426d875c386b356a6fc5702
2,002dfbc3c073b55a64a4abab34c0ca1f13897f1c
3,003998bc33cddeba02428a43391c6716e523c8f7
4,0042d2027dfa0340e31d2aa875c4be229730efb7


## Step 0.4: Record the indices of users
After having the users for the test set and the original ratings matrix, let's create some indices which we will use later on to filter data. Don't worry if it does not seem much intuitive in the beginning, you will further return to this and get a better grasp.

In [69]:
def get_indices_in_ratings_for_test_users_in_data(data: pd.DataFrame, test_users_in_data: pd.DataFrame):
    """Returns the index of the ratings matrix for the test users for which we have training data.
    
    Args:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
        test_users_in_data (pd.DataFrame): DataFrame containing the list of test users for which we have training data.
        
    Returns:
        indices_ratings_tests_users_in_data (np.array): Indices of users in test set with training data for ratings matrix.
    
    """
    unique_users = get_unique_users(data).iloc[:, 0]
    indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.to_numpy()                                                                                       
    return indices_ratings_tests_users_in_data

indices_ratings_tests_users_in_data = get_indices_in_ratings_for_test_users_in_data(data, test_users_in_data)

In [70]:
len(indices_ratings_tests_users_in_data)

700

In [71]:
print(f"As expected, the length of the indices should be {len(indices_ratings_tests_users_in_data)}, matching the number of users in test set with training data.")

As expected, the length of the indices should be 700, matching the number of users in test set with training data.


## Step 1: Understand your data flow
In the end, we will be predicting over a test set of users, which may or may not have ratings available in our training data. For the ones in the test set for which we have data in the training, we may use personalized recommendation systems. Otherwise, we need to switch to non-personalized recommendations. 

To split into train and validation, we undersample the original list of ratings but we need to make sure that the overall shape is the same (making sure that by undersampling we do not affect the shape of the full matrix).

In [72]:
def make_ratings(data: pd.DataFrame, shape: tuple = None) -> csr_matrix:
    """Creates the ratings matrix of listening history with optional shape
    
    2nd version of the make_ratings which handles the previous TODO.
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  Listening history for the users.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from data argument.
        
    Returns:
        ratings (csr_matrix): Ratings matrix with shape (n_users, n_items).
    
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    if shape == None:
        shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    return R_


R = make_ratings(data)
R

<7526x41194 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [73]:
# Percentage of listening history used for validation.
test_size = 0.2

def make_train_val_split(data: pd.DataFrame, test_size : float = 0.2, shape: tuple = None):
    """Split the data into train and validation and returns the ratings matrixes accordingly.
    
    Args:
        data (pd.DataFrame): Listening history for the users.
        test_size (float): Percentage of listening history used for validation.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from 'data' argument.
    
    Returns:
        ratings_train (csr_matrix): Ratings matrix for train.
        ratings_val (csr_matrix): Ratings matrix for validation.
    
    """
    train_data, val_data = train_test_split(data, test_size=test_size)
    R_train = make_ratings(train_data, shape=shape)
    R_val = make_ratings(val_data, shape=shape)
    return R_train, R_val

ratings_train, ratings_val = make_train_val_split(data, test_size=test_size, shape=R.shape)

In [74]:
# After the train/validation split, let's compare the number of ratings available in each matrix.
print(f"After the split we have {ratings_train.nnz:,} ratings in the train set and {ratings_val.nnz:,} ratings in the validation set.")

After the split we have 80,000 ratings in the train set and 20,000 ratings in the validation set.


## Step 2: Non-Personalized
Let's build our baseline using one of the the Non-Personalized techniques we learned in BLU10 - "Popular Items". This basically fetches the items which have more ratings overall (we don't care if it is good or bad). Is this how it should be?

<img src="./media/fry_good_bad.jpg" alt="Confused Fry" width="300"/>


Probably not. But it is worth a shot!

In [75]:
def get_most_rated(ratings: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items in a ratings matrix.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    
    def count_ratings(R_):
        return np.array(R_.sum(axis=0))[0]
    
    ratings_ = count_ratings(ratings)
    return np.negative(ratings_).argsort()[:n]


non_pers_most_rated = get_most_rated(ratings_train, 100)
non_pers_most_rated

array([ 2319,  1223, 26674,  1365,  8345, 29519,  6316, 20026,  4572,
        2332, 22545, 27894, 28340, 17635, 28759,  5011, 11402, 28238,
       30429, 16258, 19541, 22493, 28720, 16216, 26909, 30836, 13086,
       14441, 16247, 19023, 20082, 20436,  5103, 28997, 34599,  1806,
       29158, 23202, 24635,  7992, 26991, 27863, 17956, 30742, 32498,
       21322, 22305,  4829, 15185, 32196,  2294, 25567,  8105, 24171,
        4192,   198, 16855, 15870, 30652, 18871, 11177,  6795, 22988,
       20391, 33221, 12316, 28645, 25274, 26770, 16540, 33058, 28335,
       34701, 20332,  3601,  4574, 32685, 19428, 35540, 24609,   821,
        9681, 34364, 15624,  7910, 23209, 31183, 17399, 27862, 17276,
       22462, 22722, 17430, 14107, 21534, 31880,  4800, 26721, 19113,
       23228], dtype=int64)

In [76]:
def convert_non_pers_recommendations_to_df(non_pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the non-personalized most rated to an DataFrame with the users and the recommendations.
    We will basically repeat the non_pers_recs array for the number of users in need.
    
    Args:
        non_pers_recs (np.array): Array of indices for the best non-personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    non_pers_df = pd.DataFrame(np.zeros((len(users_to_pred), 1), dtype=non_pers_recs.dtype) + non_pers_recs)
    non_pers_df = pd.concat([users_to_pred, non_pers_df], axis=1)
    non_pers_df = non_pers_df.set_index("users to recommend songs")
    
    return non_pers_df


non_pers_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, unique_users_training_data)
non_pers_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
002511b392561fc1d426d875c386b356a6fc5702,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
003998bc33cddeba02428a43391c6716e523c8f7,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
0042d2027dfa0340e31d2aa875c4be229730efb7,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228


In [77]:
def create_dict_preds(preds_df: pd.DataFrame) -> dict:
    """Convert the predictions DataFrame (index:users -> columns: items) to a dictionary of key (user->list of items).
    
    Args: 
        preds_df (pd.DataFrame): DataFrame containing the users and the ordered predictions.
        
    Returns:
        preds_dict (dict): Dict of (user_id: list of items) used for evaluating the performance.
    
    """
    return {preds_df.index[i]: preds_df.values[i].tolist() for i in range(len(preds_df))}


non_pers_dict = create_dict_preds(non_pers_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(non_pers_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [2319,
  1223,
  26674,
  1365,
  8345,
  29519,
  6316,
  20026,
  4572,
  2332,
  22545,
  27894,
  28340,
  17635,
  28759,
  5011,
  11402,
  28238,
  30429,
  16258,
  19541,
  22493,
  28720,
  16216,
  26909,
  30836,
  13086,
  14441,
  16247,
  19023,
  20082,
  20436,
  5103,
  28997,
  34599,
  1806,
  29158,
  23202,
  24635,
  7992,
  26991,
  27863,
  17956,
  30742,
  32498,
  21322,
  22305,
  4829,
  15185,
  32196,
  2294,
  25567,
  8105,
  24171,
  4192,
  198,
  16855,
  15870,
  30652,
  18871,
  11177,
  6795,
  22988,
  20391,
  33221,
  12316,
  28645,
  25274,
  26770,
  16540,
  33058,
  28335,
  34701,
  20332,
  3601,
  4574,
  32685,
  19428,
  35540,
  24609,
  821,
  9681,
  34364,
  15624,
  7910,
  23209,
  31183,
  17399,
  27862,
  17276,
  22462,
  22722,
  17430,
  14107,
  21534,
  31880,
  4800,
  26721,
  19113,
  23228]}

### Step 2.2 - Creating the Ground Truth
Since we are splitting the data into train and validation, we will test the predictions we get with the training data in the validation set. To do this, we need to determine what is the ground truth (the actual outcomes) of the recommendations for the validation set. 

The main idea is to pick up a sorted array for each user on the songs with highest value in the validaton ratings matrix.

In [78]:
def get_y_true(R_val_, users_to_pred, n=100):
    """Get the ground truth (best recommendations) of the users in the validation set.
    
    Args:
        R_val_ (csr_matrix): Validation set ratings matrix.
        n (int): Number of top-n items.
        
    Returns:
        y_true_df (pd.DataFrame): DataFrame which returns the y_true items.
        
    """
    top_from_R_val = pd.DataFrame(np.negative(R_val_).toarray().argsort()[:, :n])
    y_true_df = pd.concat([users_to_pred, top_from_R_val], axis=1)
    y_true_df = y_true_df.set_index("users to recommend songs")
    return y_true_df


y_true_df = get_y_true(ratings_val, unique_users_training_data, n=100)
y_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,4131,4770,11736,5988,0,27461,27462,27463,27464,27465,...,27379,27387,27389,27390,27391,27392,27393,27394,27395,27388
002511b392561fc1d426d875c386b356a6fc5702,11972,4500,1655,4041,7737,27459,27460,27461,27462,27463,...,27387,27388,27389,27390,27391,27392,27393,27394,27395,27380
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,9901,2890,433,6966,7328,8275,0,27463,27464,27465,...,27390,27383,27391,27393,27394,27395,27396,27397,27398,27399
003998bc33cddeba02428a43391c6716e523c8f7,12236,6767,5927,12286,0,27462,27463,27464,27465,27466,...,27381,27389,27391,27392,27393,27394,27395,27396,27397,27390
0042d2027dfa0340e31d2aa875c4be229730efb7,12272,7695,27460,27461,27462,27463,27464,27465,27466,0,...,27379,27387,27389,27390,27391,27392,27393,27394,27395,27388


In [79]:
# Create the dictionary with the ground truth.
y_true_dict = create_dict_preds(y_true_df)

## Evaluate
We will use the Mean Average Precision @ K to evaluate our predictions.

In [80]:
evaluate(y_true_dict, non_pers_dict)

5.415941841236545e-06

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [81]:
# Join both dataframes with user_id's
all_test_users = pd.concat([test_users_in_data, test_users_not_in_data]).reset_index(drop=True)

In [82]:
non_pers_test_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, all_test_users)
print(non_pers_test_most_rated_df.shape)
non_pers_test_most_rated_df.head()

(1000, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
991411f0dca94f348c7bd3eae93b6e6c061605f1,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
323fbb28144eefa3eabfa22bd310dfb0713de80d,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
55c750f0951ca1021b26c0e758660bb8a2c49d3a,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
b458e3d697276a93aa6926caf1ff08e875933940,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228


In [95]:
def save_predictions(predictions: pd.DataFrame, output_path: str):
    """Save predictions to csv.
    
    Saves the predictions into a csv file with the format we need.
    We keep the index since it contains the user ids.
    
    Args:
        predictions (pd.DataFrame): DataFrame with user_id as index and ordered recommendations in the columns.
        output_path (str): Filepath for the predictions file.
    
    """
    predictions.to_csv(output_path, header=None)
    print(f"Saved to csv in '{output_path}'.")
    
    
save_predictions(non_pers_test_most_rated_df, os.path.join("data", "test_non_personalized_recommendations.csv"))

Saved to csv in 'data\test_non_personalized_recommendations.csv'.


In [97]:
# Filter the non-personalized recommendations for the users without training data and save
new_users_recommendations = non_pers_most_rated_df.iloc[index_users_not_in_data]
save_predictions(new_users_recommendations, os.path.join("data", "new_users_non_personalized.csv"))

Saved to csv in 'data\new_users_non_personalized.csv'.


## Step 3: Personalized


### Step 3.1 - Collaborative filtering


In [98]:
def make_user_similarities(R_):
    """Creates the user similarities matrix.
    
    Args:
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        user_similarities (csr_matrix): Matrix with user similarities.
    
    """
    return cosine_similarity(R_, dense_output=False)


user_similarities = make_user_similarities(ratings_train)
user_similarities

<7526x7526 sparse matrix of type '<class 'numpy.float64'>'
	with 1075960 stored elements in Compressed Sparse Row format>

In [99]:
def make_user_predictions_collab_filt(S: csr_matrix, R_: csr_matrix):
    """Predict using collaborative filtering.
    
    Args:
        S (csr_matrix): Similarities matrix (tipically using the cosine_similarity).
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        preds (csr_matrix): Predictions matrix.
    
    """
    weighted_sum = np.dot(S, R_)
    
    # We use the absolute value to support negative similarities.
    # In this particular example there are none.
    sum_of_weights = np.abs(S).sum(axis=1)
    
    preds = weighted_sum / sum_of_weights
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return csr_matrix(preds)
 

collab_filt_user_preds = make_user_predictions_collab_filt(user_similarities, ratings_train)
collab_filt_user_preds

<7526x41194 sparse matrix of type '<class 'numpy.float64'>'
	with 9196705 stored elements in Compressed Sparse Row format>

In [100]:
def sparsity(matrix):
    """Calculates the sparsity of a matrix.
    
    Args:
        matrix (csr_matrix): Sparse matrix.
        
    Returns:
        sparsity_ (float): Sparsity percentage (between 0 and 1).
    
    """
    return 1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])


sparsity(collab_filt_user_preds)

0.9703357018612282

In [101]:
def get_most_rated_from_user_preds(user_preds_: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items from the user predictions.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    pred_ = np.negative(user_preds_).toarray()
    return pred_.argsort()[:, :n]


collab_filt_most_rated = get_most_rated_from_user_preds(collab_filt_user_preds, 100)
print(collab_filt_most_rated.shape)
collab_filt_most_rated

(7526, 100)


array([[10121, 14230, 33221, ...,   197, 17920, 12780],
       [27674, 21943, 12236, ..., 25001, 31479,  2888],
       [ 8636,  1223, 26926, ...,  3459,  3099, 35504],
       ...,
       [32735, 19836, 13622, ..., 27392, 27393, 27394],
       [ 5397,  1806, 20366, ...,  3297, 22649, 28645],
       [30953, 24621, 27336, ..., 31337, 26804, 25344]], dtype=int64)

In [102]:
def convert_pers_recommendations_to_df(pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """Converts the personalized most rated to an DataFrame with the users and the recommendations.
    
    Args:
        pers_recs (np.array): Array of indices for the best personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    pers_df = pd.concat([users_to_pred, pd.DataFrame(pers_recs)], axis=1)
    pers_df = pers_df.set_index("users to recommend songs")
    
    return pers_df


collab_filt_most_rated_df = convert_pers_recommendations_to_df(collab_filt_most_rated, unique_users_training_data)
collab_filt_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,10121,14230,33221,2319,8951,11177,26827,11382,19541,28720,...,30742,3504,26770,1529,35429,4675,10225,197,17920,12780
002511b392561fc1d426d875c386b356a6fc5702,27674,21943,12236,4667,12565,7689,7968,30158,33286,12855,...,4354,3246,31880,12895,5648,4800,21769,25001,31479,2888
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,8636,1223,26926,8345,29519,20026,28759,20359,24511,6316,...,28363,4637,3601,35033,30555,16540,27405,3459,3099,35504
003998bc33cddeba02428a43391c6716e523c8f7,33434,28910,3706,4108,10578,23931,28340,33236,21534,26674,...,1042,3372,7218,18684,23395,26183,20436,11696,25227,9545
0042d2027dfa0340e31d2aa875c4be229730efb7,3825,3099,25574,22545,27185,20834,31095,19937,3652,11077,...,22801,24584,4488,13086,16082,33601,4183,10464,22845,35057


In [103]:
collab_filt_dict = create_dict_preds(collab_filt_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(collab_filt_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [10121,
  14230,
  33221,
  2319,
  8951,
  11177,
  26827,
  11382,
  19541,
  28720,
  17635,
  15289,
  26674,
  28922,
  32685,
  28340,
  29158,
  5298,
  11923,
  4829,
  30718,
  30836,
  1054,
  28363,
  34705,
  31510,
  1365,
  9856,
  35236,
  2370,
  1223,
  3181,
  13034,
  8345,
  12739,
  22462,
  20746,
  24171,
  33594,
  2294,
  19547,
  28232,
  29076,
  28774,
  20026,
  18329,
  30894,
  25635,
  22764,
  6316,
  3601,
  30000,
  25078,
  34901,
  11040,
  20989,
  17345,
  22173,
  16247,
  16258,
  19110,
  25161,
  4469,
  29583,
  27894,
  5011,
  8340,
  350,
  13243,
  15529,
  12610,
  26027,
  24325,
  6507,
  27544,
  14420,
  28997,
  21630,
  32104,
  30427,
  33601,
  13657,
  32564,
  23819,
  16852,
  7992,
  8292,
  19188,
  8323,
  33526,
  30742,
  3504,
  26770,
  1529,
  35429,
  4675,
  10225,
  197,
  17920,
  12780]}

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [104]:
evaluate(y_true_dict, collab_filt_dict)

0.031279727852493035

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [105]:
# Filter the collaborative filtering most rated DataFrame  using the test_users_in_data mask.
collab_filt_most_rated_in_data_df = collab_filt_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(collab_filt_most_rated_in_data_df.shape)
collab_filt_most_rated_in_data_df.head()

(700, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,22545,6921,14489,9543,20711,20316,28720,17821,12751,34778,...,27475,27500,27463,27486,27388,27487,27488,27489,27490,27491
991411f0dca94f348c7bd3eae93b6e6c061605f1,29909,21997,1237,24645,18954,35254,30988,4566,9982,35318,...,9923,22765,157,15132,22021,27405,27486,27485,27470,27407
323fbb28144eefa3eabfa22bd310dfb0713de80d,22573,22222,2803,4793,34305,7011,21688,28503,17983,25021,...,7065,16578,26527,25790,19229,11402,18797,20865,5416,10499
55c750f0951ca1021b26c0e758660bb8a2c49d3a,2319,28759,2332,1223,4761,28340,26059,32498,16005,30836,...,1471,22649,28051,28238,19099,14053,5125,21071,1230,27832
b458e3d697276a93aa6926caf1ff08e875933940,8345,30836,15332,24635,30427,26521,20095,21944,8819,25106,...,1990,9319,2146,29934,15456,17186,10324,30542,29089,10795


In [106]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(300, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
002511b392561fc1d426d875c386b356a6fc5702,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
003998bc33cddeba02428a43391c6716e523c8f7,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
005b1fab38cdeb9d5bb97debcf73b44050994a3e,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
005cc5d858319f13f88228f62341e5a4270f8e75,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
006d3c79b9ed677280f8ddbc422d7b0fedd6d1fa,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228


In [107]:
collab_filt_most_rated_test_df = pd.concat([collab_filt_most_rated_in_data_df, new_users_recommendations])
print(collab_filt_most_rated_test_df.shape)

(1000, 100)


In [108]:
# Save the collaborative filtering recommendations 
save_predictions(collab_filt_most_rated_test_df, os.path.join("data", "collab_filt_recommendations.csv"))

Saved to csv in 'data\collab_filt_recommendations.csv'.


### Step 3.2 - Content-based recommendations

In [109]:
# If you had a FileNotFoundError, make sure you have unziped the `song_tag.zip` file.
def read_tags():
    """Import the song tags file.
    
    Returns:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        
    """
    path = os.path.join('data', 'song_tag.csv')
    data = pd.read_csv(path)
    return data


tags = read_tags()
print(tags.shape)
tags.head()

(4909471, 3)


,song_index,tag_index,val
0,254229,206,100.0
1,254229,1125,66.0
2,254229,582,66.0
3,254229,914,33.0
4,254229,95,33.0


In [110]:
def read_songs():
    """Import the songs file.
    
    Returns:
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
    
    """
    path = os.path.join('data', 'songs.txt')
    data = pd.read_csv(path, names=['song_id', 'song_index'], sep=' ')
    return data


songs= read_songs()
print(songs.shape)
songs.head()

(386213, 2)


,song_id,song_index
0,SOAAADD12AB018A9DD,1
1,SOAAADE12A6D4F80CC,2
2,SOAAADF12A8C13DF62,3
3,SOAAADZ12A8C1334FB,4
4,SOAAAFI12A6D4F9C66,5


In [111]:
def merge_tags_songs(tags, songs, data):
    """Join the tags, songs and ratings data sources into a single frame).
    
    Args:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
        
    Returns:
        tags_cleaned (pd.DataFrame): Matches the song_id to the tag_index with a corresponding value.
    
    """
    tags_cleaned = (tags.merge(songs, how='left', on='song_index')
                        .merge(data, how='right', on='song_id')
                   )[['song_id', 'tag_index', 'val']]
    return tags_cleaned

tags_cleaned = merge_tags_songs(tags, songs, data)
print(tags_cleaned.shape)
tags_cleaned.head()

(4773486, 3)


,song_id,tag_index,val
0,SODDEQU12AAF3B2FC8,104.0,100.0
1,SODDEQU12AAF3B2FC8,696.0,42.0
2,SODDEQU12AAF3B2FC8,20425.0,42.0
3,SODDEQU12AAF3B2FC8,51057.0,28.0
4,SODDEQU12AAF3B2FC8,324.0,28.0


In [112]:
# Get the number of unique song_id's for which we have tags. 
tags_cleaned.song_id.nunique()

41194

In [113]:
def make_item_profiles(data):
    """Creates the item profiles matrix.
    
    Args:
        data (pd.DataFrame): DataFrame containing the (rows, columns, values) for the ratings matrix.
                             In this case, we have (song_id, tag_index, value).
    
    Returns:
        item_profiles (csr_matrix): Item profiles matrix. Items as rows, tags as columns.
    
    """
    items, item_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    tags, tag_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(items), len(tags))

    item_profiles_ = coo_matrix((values, (item_pos, tag_pos)), shape=shape).tocsr()
    return item_profiles_


item_profiles = make_item_profiles(tags_cleaned)
item_profiles

<41194x219644 sparse matrix of type '<class 'numpy.float64'>'
	with 1364190 stored elements in Compressed Sparse Row format>

In [114]:
def make_user_profiles(R_, item_profiles_):
    """Calculate the user profiles with the items.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        
    Returns:
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
        
    """
    return np.dot(R_, item_profiles_)


user_profiles = make_user_profiles(ratings_train, item_profiles)
user_profiles

<7526x219644 sparse matrix of type '<class 'numpy.float64'>'
	with 2021240 stored elements in Compressed Sparse Row format>

In [115]:
def make_user_predictions_content(R_, item_profiles_, user_profiles_):
    """Produces content-based predictions.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
       
    Returns:
        preds (csr_matrix): Predictions of ratings using content-based recommendations.
    
    """
    
    # Since we are not looking at the values inbetween the cosine_similarity and excluding previously rated items,
    # we can define dense_output=False to speed up.
    preds = cosine_similarity(user_profiles_, item_profiles_, dense_output=False)
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return preds


content_user_preds = make_user_predictions_content(ratings_train, item_profiles, user_profiles)
content_user_preds

C:\Users\afrodrigues\AppData\Local\Continuum\anaconda3\envs\blu12_1\lib\site-packages\scipy\sparse\_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


<7526x41194 sparse matrix of type '<class 'numpy.float64'>'
	with 197293126 stored elements in Compressed Sparse Row format>

In [116]:
content_most_rated = get_most_rated_from_user_preds(content_user_preds, 100)
print(content_most_rated.shape)
content_most_rated

(7526, 100)


array([[11216, 31075, 14867, ..., 40011, 16911, 35072],
       [ 4523, 10705, 26150, ..., 38695, 12802, 23538],
       [22203, 29133, 23699, ..., 16192,  2750,  9596],
       ...,
       [25973, 32433, 19037, ..., 34844, 30998, 20068],
       [ 5829, 18830, 11783, ..., 24034, 25022, 22863],
       [33628, 35170, 10040, ..., 26093, 16221,  4811]], dtype=int64)

In [117]:
content_most_rated_df = convert_pers_recommendations_to_df(content_most_rated, unique_users_training_data)
print(content_most_rated_df.shape)
content_most_rated_df.head()

(7526, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,11216,31075,14867,14804,13476,16909,31247,1652,24379,8364,...,35139,18209,36343,20216,3064,33279,31001,40011,16911,35072
002511b392561fc1d426d875c386b356a6fc5702,4523,10705,26150,8954,19849,18242,25396,33378,37251,24785,...,17416,36951,40770,34687,10037,23431,37456,38695,12802,23538
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,22203,29133,23699,23689,5573,27000,39443,36411,25774,3817,...,24582,24544,40697,16643,27063,19858,23415,16192,2750,9596
003998bc33cddeba02428a43391c6716e523c8f7,24952,22418,9990,1652,12668,17539,36582,13571,38696,24379,...,1602,7602,21888,7330,23990,5578,10084,40384,32331,36635
0042d2027dfa0340e31d2aa875c4be229730efb7,15388,4507,12684,34724,27657,34347,21967,14458,31298,33576,...,16986,4213,13497,32609,16061,745,3404,39845,31013,22999


In [118]:
content_dict = create_dict_preds(content_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(content_dict.items())[0:2])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [11216,
  31075,
  14867,
  14804,
  13476,
  16909,
  31247,
  1652,
  24379,
  8364,
  10067,
  24792,
  39853,
  15960,
  34946,
  25069,
  32331,
  29968,
  31108,
  25140,
  18862,
  15627,
  24952,
  9338,
  37888,
  12868,
  21866,
  35105,
  2291,
  36092,
  7508,
  27583,
  39289,
  28563,
  33204,
  22123,
  21906,
  40475,
  21368,
  9990,
  29948,
  31962,
  1350,
  36582,
  10889,
  19418,
  23211,
  2848,
  1918,
  38696,
  7330,
  27103,
  7542,
  30837,
  18994,
  31321,
  9143,
  17560,
  26439,
  13067,
  30324,
  28164,
  19126,
  3262,
  16276,
  13571,
  14794,
  3369,
  13023,
  3714,
  22418,
  17539,
  10414,
  37145,
  7602,
  23156,
  38828,
  34500,
  32154,
  15209,
  25861,
  37391,
  8848,
  37596,
  24339,
  2381,
  10458,
  23909,
  1762,
  40692,
  35139,
  18209,
  36343,
  20216,
  3064,
  33279,
  31001,
  40011,
  16911,
  35072],
 '002511b392561fc1d426d875c386b356a6fc5702': [4523,
  10705,
  26150,
  895

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [119]:
evaluate(y_true_dict, content_dict)

0.0017902152206683876

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [120]:
content_most_rated_in_data_df = content_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(content_most_rated_in_data_df.shape)
content_most_rated_in_data_df.head()

(700, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,26229,5497,6678,31292,30880,35938,9247,5995,11625,31345,...,10675,4393,27089,23083,7177,11482,21520,17048,14136,7157
991411f0dca94f348c7bd3eae93b6e6c061605f1,20339,20414,23196,37026,1590,1306,16975,40677,27445,33253,...,37030,36152,21211,16872,14668,28150,14819,26621,22746,24727
323fbb28144eefa3eabfa22bd310dfb0713de80d,17416,8954,24676,29263,28562,14414,32107,8087,8445,6692,...,38367,38262,19196,27721,5727,21581,13513,32918,10643,21605
55c750f0951ca1021b26c0e758660bb8a2c49d3a,15266,2255,35209,16691,4393,18415,35109,7889,14667,40422,...,5618,31288,1772,11960,33763,22875,20863,29435,6915,25260
b458e3d697276a93aa6926caf1ff08e875933940,18818,6317,3013,35286,26266,33776,30660,21501,38774,7864,...,10293,29952,21358,7164,16211,28956,15713,14081,20983,13845


In [121]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(300, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
002511b392561fc1d426d875c386b356a6fc5702,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
003998bc33cddeba02428a43391c6716e523c8f7,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
005b1fab38cdeb9d5bb97debcf73b44050994a3e,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
005cc5d858319f13f88228f62341e5a4270f8e75,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228
006d3c79b9ed677280f8ddbc422d7b0fedd6d1fa,2319,1223,26674,1365,8345,29519,6316,20026,4572,2332,...,22462,22722,17430,14107,21534,31880,4800,26721,19113,23228


In [123]:
content_most_rated_test_df = pd.concat([content_most_rated_in_data_df, new_users_recommendations])
print(content_most_rated_test_df.shape)

(1000, 100)


In [124]:
# Save the content-based filtering recommendations 
save_predictions(content_most_rated_test_df, os.path.join("data", "content_recommendations.csv"))

Saved to csv in 'data\content_recommendations.csv'.
